In [1]:
!git clone https://github.com/jishengpeng/WavTokenizer
%cd WavTokenizer
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml

Cloning into 'WavTokenizer'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 200 (delta 65), reused 37 (delta 35), pack-reused 84 (from 1)
Receiving objects: 100% (200/200), 469.31 KiB | 4.79 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/kaggle/working/WavTokenizer
--2025-05-12 20:29:55--  https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt
Resolving huggingface.co (huggingface.co)... 18.244.202.68, 18.244.202.118, 18.244.202.60, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.68|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f8/d8/f8d8b97f33126a1e3a4c3ffe2e6af86c26776cfa33aee1294672329b62115562/078d11581aa10cc91572bfcff7ef00b71d8e24c4c359e98f9194a1a9d21ae8a8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27

In [2]:
from encoder.utils import convert_audio
import torchaudio
import torch
from decoder.pretrained import WavTokenizer


device=torch.device('cuda')

config_path = "/kaggle/working/WavTokenizer/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml"
model_path = "/kaggle/working/WavTokenizer/wavtokenizer_medium_music_audio_320_24k_v2.ckpt"

wavtokenizer = WavTokenizer.from_pretrained0802(config_path, model_path)
wavtokenizer = wavtokenizer.to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/kaggle/working/WavTokenizer/decoder/pretrained.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True

## Groove dataset

In [3]:
import os

wav_files = []

for root, dirs, files in os.walk('/kaggle/input/groove1/groove/'):
    for file in files:
        if file.endswith('.wav'):
            wav_files.append(os.path.join(root, file))

In [4]:
! rm -rf /kaggle/working/groove_original
! rm -rf /kaggle/working/groove_decoded
! mkdir /kaggle/working/groove_original
! mkdir /kaggle/working/groove_decoded

In [5]:
from tqdm import tqdm
import shutil

for wav_file in tqdm(wav_files):
    name = wav_file.split('/')[-1]
    dst = os.path.join('/kaggle/working/groove_original/', name)
    shutil.copyfile(wav_file, dst)

100%|██████████| 1090/1090 [01:28<00:00, 12.36it/s]


In [6]:
import os
import gc

pt_files = []

for root, dirs, files in os.walk('/kaggle/input/groove-tokens'):
    for file in files:
        if file.endswith('.pt'):
            pt_files.append(os.path.join(root, file))

In [7]:
def save_wav_from_pt(pt_name, name):
    with torch.no_grad():
        audio_tokens = torch.load(pt_name, map_location=device)
        audio_tokens = audio_tokens[:, :, :15000]
        features = wavtokenizer.codes_to_features(audio_tokens)
        bandwidth_id = torch.tensor([0], device=device)  
        audio_out = wavtokenizer.decode(features, bandwidth_id=bandwidth_id)
        torchaudio.save(name, audio_out.detach().cpu(), sample_rate=24000, encoding='PCM_S', bits_per_sample=16)
    del audio_tokens
    del features
    del bandwidth_id
    del audio_out

    torch.cuda.empty_cache()
    gc.collect()

In [8]:
for pt_file in tqdm(pt_files):
    name = pt_file.split('/')[-1][:-2] + 'wav'
    dst = os.path.join('/kaggle/working/groove_decoded/', name)
    save_wav_from_pt(pt_file, dst)

  0%|          | 0/1090 [00:00<?, ?it/s]/tmp/ipykernel_19/3358673782.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  audio_tokens = torch.load(pt_name, map_location=devi

In [9]:
shutil.make_archive('groove_decoded', 'zip', '/kaggle/working/groove_decoded/')
shutil.make_archive('groove_original', 'zip', '/kaggle/working/groove_original/')

'/kaggle/working/WavTokenizer/groove_original.zip'